## Library import

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
#import plotly.express.colors as pc
pd.options.plotting.backend = 'plotly'
from sklearn.preprocessing import minmax_scale, MinMaxScaler
from sklearn.decomposition import PCA
from scipy.signal import savgol_filter

In [2]:
import plotly.io as pio
pio.templates["standard"] = go.layout.Template(layout=dict(
    title_font_family = 'Times New Roman',
    font_family = 'Times New Roman',
    legend_font_family = 'Times New Roman',
    font_size = 22,
    margin = dict(t=5, b=5, l=50, r=5),
    xaxis_mirror= True,
    yaxis_mirror= True,
    yaxis_nticks = 4,
    xaxis_nticks = 5,
    legend_title = '',
    #legend_font_family = 'Courier',
    legend_orientation = 'v',
    legend_xanchor = 'right',
    legend_bgcolor = 'rgba(256,256,256,0.8)',
    legend_x = 1,
    legend_y = 1.15,
    width = 1000, 
    height = 500))
pio.templates.default = 'simple_white+standard'

In [4]:
df = pd.read_csv("Amit.txt", sep='\t', decimal='.', header=None)
df

,0,1
0,1.710,1.529
1,2.294,1.529
2,3.824,2.162
3,3.824,2.417
4,3.900,2.756
...,...,...
348,NaN,238.500
349,NaN,239.967
350,NaN,239.999
351,NaN,241.586


In [23]:
fig = go.Figure()
fig.add_trace(go.Violin(x=[np.log10(x) for x in df.iloc[:,0]], name = '', 
                            line_color = 'blue',  #np.log10(df.loc[:,column]
                            scalemode='width', 
                            meanline_visible=True,
                            scalegroup = 'A', side='positive', 
                            points='all',
                            jitter = 0.6))
fig.add_trace(go.Violin(x=[np.log10(x) for x in df.iloc[:,1]], name = '', 
                            line_color = 'red',  #np.log10(df.loc[:,column]
                            scalemode='width', 
                            meanline_visible=True,
                            scalegroup = 'A', side='positive', 
                            points='all',
                            jitter = 0.6))
fig.update_layout(xaxis_title = 'SWCNT bundle diameter, lg(nm)')
fig.show()

## Necessary functions

In [ ]:
def load_data(gas: str = 'NO2'):
    gas_path_dict = {'NO2': r"C:\Users\zamba\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Feb 2021\NO2_15_02_protocol10-15-25.brotli",
                     'H2S': r"C:\Users\zamba\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Feb 2021\H2S_17_02_protocol10-15-25.brotli",
                     'Acet': r"C:\Users\zamba\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Feb 2021\Acet_19_02_protocol10-15-25.brotli"}
    df = pd.read_parquet(gas_path_dict[gas])
    if gas == 'H2S':
        df = df[15*402:]  # first 15 pulses are cut
    elif gas == 'NO2':
        df = df[:-400]  #df = df[40*402:-400]  # - last pulse missing or rather last 2 datapoints are missing
    else:
          # pass df = df[40*402:]  # last meas_cycle has only 79*402 records instead of 120*402
        pass
    switch_mask = (df['MFC_target'] == 0) & (df['MFC_target'].shift() == 25)
    incremental_values = switch_mask.cumsum()
    return df.assign(meas_cycle = incremental_values)

def extract_gas_protocol(df: pd.DataFrame) -> (np.array, np.array):
    try:
        dfg = df.copy(deep=True)
        dfg = dfg.assign(new_gas = dfg['MFC_target'].rolling(window=3).apply(lambda x: np.NaN if x[0] == x[1] == x[2] else x[1], raw = True))
        dfg = dfg.dropna()
        return [df.index[0]] + dfg.index.to_list() + [df.index[-1]], [df.MFC_target.iloc[0]] + dfg.new_gas.to_list() + [df.MFC_target.iloc[-1]]
    
    except AttributeError:
        print("****************\nTime and MFC_target are not in df.columns\n****************")

def quick_I_plot(df: pd.DataFrame, column: str = "I", step: int = 6):
    slice_ = slice(0,df.shape[0],step)
    x = df.index[slice_]
    y_ = df.loc[:,column]
    y = y_.iloc[slice_]
    x_2, y_2 = extract_gas_protocol(df)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scattergl(x = x, y = y, mode = 'lines', name = 'raw data', line_color = px.colors.qualitative.Plotly[0]), secondary_y = False)
    fig.add_trace(go.Scattergl(x = x_2, y = y_2, mode = 'lines', name = 'analyte conc', line_color = px.colors.qualitative.Plotly[1]), secondary_y = True)
    fig.update_layout(yaxis_title_text = 'Current, A', 
                      yaxis2_title_text = f'{target_gas_dict[gas]} concentration, ppm',
                      yaxis2_fixedrange = True,
                      title_text = f'{target_gas_dict[gas]}',  # data_list[filenumber][:-7]
                      width = 1100, height = 500, hovermode = False)
    return fig

target_gas_dict = dict(H2S = "H<sub>2</sub>S",
                       NO2 = "NO<sub>2</sub>",
                       NH3 = "NH<sub>3</sub>",
                       H2O = "H<sub>2</sub>O",
                       Acet = "Acetone",
                       EtOH = "EtOH",
                       CO = "CO")

def true_gas(df: pd.DataFrame):
    #errors = df.flow_target_error.abs().values
    return df.MFC_target.values - df.flow_target_error.values
def SavGol(trace, window_length: int = 600, polyorder: int = 3) -> np.ndarray:
    baseline_trend = savgol_filter(x = trace, window_length = window_length, polyorder = polyorder, mode = 'mirror')
    return trace - baseline_trend

def cut_into_Vpulses(data_column: np.array, gas_column: np.array, meas_cycle_column: np.array, gas: str, dp_per_pulse: int = 402): 
    gas_column = gas_column.reshape((len(data_column)//dp_per_pulse, dp_per_pulse)).mean(axis=1)
    meas_cycle = meas_cycle_column[::dp_per_pulse]
    if len(data_column) % dp_per_pulse != 0:
        print(f"df shape is not divisible by {dp_per_pulse}\n{len(data_column) % dp_per_pulse}")
    else:
        datadf = data_column.reshape((len(data_column)//dp_per_pulse, dp_per_pulse))
        zeros = np.zeros(len(data_column)//dp_per_pulse)
        datadf = pd.DataFrame(datadf).assign(NO2 = zeros,
                                             H2S = zeros,
                                             Acet = zeros, 
                                             meas_cycle = meas_cycle)
        datadf.loc[:,gas] = gas_column
        return datadf.assign(gas=gas)
def dedrift_n_cut(df, how: str, window_length, gas):
    '''how - choose one of multienv, topenv, full to dedrift accordingly. Any other value will not dedrift data at all'''
    data = df.I.values
    gas_values = true_gas(df)
    meas_cycle_column = df.meas_cycle.values
    # dedrifting
    if how == 'full':
        dedrifted = SavGol(data, window_length=window_length * 402)
    elif how == 'multienv':
        env_dict = {}
        envelopes = [50, 150, 201, 300]
        for envelope in envelopes:
            env_dict[envelope] = savgol_filter(data[envelope::402], window_length=window_length, polyorder=3,mode = 'mirror')
        envelope_ = np.array([[x]*402 for x in pd.DataFrame(env_dict).values.mean(axis=1)]).flatten()
        dedrifted = data - envelope_
    elif how == 'topenv':
        envelope_ = np.array([[x]*402 for x in savgol_filter(data[201::402], window_length=window_length, polyorder=3, mode = 'mirror')]).flatten()
        dedrifted = data - envelope_
    else: 
        #print("Data not dedrifted")
        dedrifted = data - data.mean()
    # cutting
    return cut_into_Vpulses(dedrifted, gas_values, meas_cycle_column, gas, dp_per_pulse=402)


In [ ]:
def choose_last_n_cycles(data: np.ndarray, n: int = 1) -> np.ndarray:
    """Takes in a ndarray of shape (samples, features), target gas column (usually, a list)"""
    if n >= 10: raise ValueError("The shortest part of the protocol contains 10 thermocycles. Try n < 10!")
    target_cycles = np.array([10, 40, 50, 80, 90, 120])
    target_cycles_ = np.array([], dtype=int)
    for j in range(10):  # 10 is the number of measurement cycles
        tc = target_cycles + 120 * j - 1
        target_cycles_ = np.append(target_cycles_, tc)
        if n > 1:
            for i in range(n-1):
                target_cycles_ = np.append(target_cycles_, tc - 1 - i)
    target_cycles_ = sorted(target_cycles_[target_cycles_ < data.shape[0]])  # masking all values that are not needed + sorting the list
    return data.iloc[sorted(target_cycles_),:]

In [ ]:
from re import match

def color_with_opacity(color: str, opacity: float = 0.2) -> str:
    '''Takes a color of any format (hex, rgb) returns rgba with given opacity'''
    # Check if the input color is in hex format
    if color.startswith("#"):
        r, g, b = px.colors.hex_to_rgb(color)
    else:
        # Check if the input color is in 'rgb(r, g, b)' format
        rgb_match = match(r'rgb\((\d+), (\d+), (\d+)\)', color)
        if rgb_match:
            r, g, b = map(int, rgb_match.groups())
        else:
            raise ValueError("Invalid color format. Supported formats: hex or 'rgb(r, g, b)'.")

    rgba_color = f'rgba({r}, {g}, {b}, {opacity})'    
    return rgba_color

def trace_with_CI(index: np.ndarray, trace: np.ndarray, std: np.ndarray, z_score: float = 1.96, fillcolor: str = 'rgb(100, 100, 100)', **kwargs) -> go.Scatter:
    '''Creates a go.Scatter object representing the trace and its confidence interval
    z_score - multiplier for std if z_score = 1 - 68%, 2 - 95.6%, 3 - 99%
    **kwargs are added for compatibility with go.Scatter'''

    x = np.append(index, index[::-1])
    std_ = std * z_score
    lower_bound = trace - std_
    upper_bound = trace + std_
    y = np.append(lower_bound, upper_bound[::-1])
    kwargs.setdefault('fillcolor', color_with_opacity(fillcolor))
    kwargs.setdefault('line_color', 'rgba(255, 255, 255, 0)')
    kwargs.setdefault('showlegend', False)
    kwargs.setdefault('name', '95% Confidence interval')
    return go.Scatter(x = x, y = y, fill='toself', hoverinfo = 'skip', **kwargs)

from scipy.stats import norm

def calculate_p_value(z_score):
    '''Calculate the cumulative probability up to the given z-score'''
    return (2 * norm.cdf(abs(z_score)) - 1) * 100


In [ ]:
def subtract_previous_row(data: pd.DataFrame):
    result = data.copy()  # Create a copy to avoid modifying the original DataFrame
    for index, row in data.iterrows():
        if index - 30 in data.index:
            result.loc[index] = row - data.loc[index - 30]
    return result

In [ ]:
def datetime_index_to_seconds(datetime_index: pd.core.indexes.datetimes.DatetimeIndex, minutes: bool = False) -> np.ndarray:
    # Calculate the time difference in seconds from the starting value
    time_difference = datetime_index - datetime_index[0]
    seconds = time_difference.total_seconds().values
    if minutes: seconds = seconds / 60
    return seconds

def print_tolerances(df: pd.DataFrame, return_str: bool = True) -> str:
    dt_s = pd.Series(datetime_index_to_seconds(df.index)).diff().values[1:]
    dt_str = f'dt = {dt_s.mean()*1000:.0f} \u00b1 {dt_s.std()*1000:.0f} ms'
    dV_s = df.V.diff()
    dV_s_ = np.append(dV_s.loc[dV_s > 0.01].values, abs(dV_s.loc[dV_s < -0.01].values))
    dV_str = f'dV = {dV_s_.mean()*1000:.1f} \u00b1 {dV_s_.std()*1000:.1f} mV'
    if return_str: 
        return dt_str + '<br>' + dV_str
    else: 
        print('', dt_str, '\n', dV_str)

colorscale_dict = dict(NO2= 'Sunset',
                       H2S= 'plasma_r',
                       Acet= 'Viridis_r')

gas = 'NO2'
def create_gas_rectangles(gas_index: list, gas_values: list, colorscale: str = colorscale_dict[gas], **kwargs) -> list:
    '''takes the index and values from extract_gas_protocol function and returns the list of go.layout.Shape dictionaries'''
    rectangles = []
    unique_gas_values = list(set(gas_values))
    colors = dict(zip(unique_gas_values, px.colors.sample_colorscale(colorscale, minmax_scale(unique_gas_values))))
    kwargs.setdefault('yref', 'paper')
    kwargs.setdefault('y0', 0)
    kwargs.setdefault('y1', 1)
    for i in range(0, len(gas_index) - 1, 2):
        text = '' if gas_values[i]==0 else f'<b>{gas_values[i]:.0f}</b>'  # gas is a global variable  #  ppm {target_gas_dict[gas]}
        color = 'rgba(255,255,250,0.4)' if gas_values[i]==0 else color_with_opacity(colors[gas_values[i]], 0.7)  # as well as target_gas_dict
        rectangle = dict(type = 'rect', 
                         xref= 'x', yref= kwargs['yref'], 
                         x0= gas_index[i], x1= gas_index[i+1], y0= kwargs['y0'], y1=kwargs["y1"], 
                         fillcolor = color, line_width= 0, 
                         label = dict(text= text, textangle= -0, xanchor= 'center', textposition= 'bottom center', padding= 20, font_color='white'),
                         layer = 'above',
                         )
        rectangles.append(go.layout.Shape(rectangle))
    return rectangles

In [ ]:
def envelope(data_: pd.DataFrame, V: float = None, Temp: float = None, calib_slice_shift: int = 0, is_mA: bool = False, **kwargs) -> go.Scatter:
    '''Takes dataframe for compatibility. Dataframe has to contain I and V columns as well as timestamp index. Returns go.Scatter of the envelope trace
    V has to be in range (5,25)
    dw = 0.003, dt= 0.7, dV= 0.1'''
    mA = 1000 if is_mA else 1
    calibration_slice = slice(402*calib_slice_shift + 0, 402*calib_slice_shift + 201,1)
    data_ = data_.assign(temp = data_.V * data_.I * 202.7 + 28)  # creating a temperature column
    tmax, tmin = data_.temp.max(), data_.temp.min()  # tmax, tmin for shorter syntax
    if Temp: 
        if Temp > tmax or Temp < tmin: 
            raise ValueError(f'Input Temperature between [{tmin:.1f}, {tmax:.1f}]')
        V_idx = (data_.temp.iloc[calibration_slice] - Temp).abs().argsort()[:1].values[0]  # getting the integer index of the temperature value closest to Temp in the given calibration_slice
    else:
        V_idx = (data_.V.iloc[calibration_slice] - V).abs().argsort()[:1].values[0]
        Temp = data_.temp.iloc[V_idx] 
    color = px.colors.sample_colorscale('Plasma', minmax_scale([tmin, Temp, tmax]))[1]
    data = data_.I.iloc[slice(V_idx, data_.shape[0], 402)]
    data.index = datetime_index_to_seconds(data.index, True)
    kwargs.setdefault('line_color', color)
    kwargs.setdefault('line_width', 3)
    kwargs.setdefault('mode', 'lines')
    kwargs.setdefault('name', f'{Temp:.0f}<sup>o</sup>C')  # {round(data_.V.iloc[V_idx],0):.1f}V, 
    trace = go.Scatter(x = data.index, y=data.values*mA, **kwargs)
    return trace

## Graphs

### Fig 2. Pyrometer calibration

In [ ]:
df = pd.read_csv(r"C:\Users\zamba\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Pyrometer and Raman\Pyrometer.txt", sep='\t', decimal=',', index_col=0)

In [ ]:
from sklearn.metrics import r2_score
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from scipy.stats import t

def Linear_regr_CI(x: np.ndarray, y: np.ndarray, alpha: float = 0.05) -> dict:
    '''Takes x, y as input, returns linear fit and boundaries for CI
    
    dict keys are r2, upper_boundary, lower_boundary, fit, slope, intercept, std_slope, std_intercept'''
    output_dict = {}
    n = len(x)
    t_critical = t.ppf(1 - alpha / 2, df= n-2)  # student's coefficient for the appropriate number of degrees of freedom
    X = add_constant(x)
    model = OLS(y, X).fit()
    y_pred = model.predict(X)
    output_dict['r2'] = r2_score(y_pred, y)
    SE = np.sqrt(sum([(yi-yi_p)**2 for yi, yi_p in zip(y, y_pred)]) / (n - 2))  # unexplained variation
    SS = sum([(xi - x.mean())**2 for xi in x])  # variance of x * len(x)
    output_dict['upper_boundary'] = [(yi_pred + t_critical*SE*np.sqrt(1/n + (xi - x.mean())**2 / SS)) for xi, yi_pred in zip(x, y_pred)]
    output_dict['lower_boundary'] = [(yi_pred - t_critical*SE*np.sqrt(1/n + (xi - x.mean())**2 / SS)) for xi, yi_pred in zip(x, y_pred)]
    output_dict['fit'] = y_pred
    output_dict['slope'] = model.params[1]
    output_dict['intercept'] = model.params[0]
    output_dict['std_slope'] = model.bse[1]
    output_dict['std_intercept'] = model.bse[0]
    return output_dict

In [ ]:
t.ppf(0.995, df=49)

In [ ]:
x = df.index.values
y = np.array(df.T.values.tolist()[0])
LR = Linear_regr_CI(x, y, 0.05)
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.append(x, x[::-1]), 
                         y= np.append(LR['upper_boundary'], LR['lower_boundary'][::-1]), 
                         fillcolor= color_with_opacity(px.colors.qualitative.Plotly[0]),
                         fill= 'toself',
                         hoverinfo='skip',
                         line_color= 'rgba(0,0,0,0)',
                         showlegend = True, name = '95% CI'))
fig.add_trace(go.Scatter(x=x, y=LR['fit'], name='linear fit', mode = 'lines', line_color = px.colors.qualitative.Plotly[0]))
fig.add_trace(go.Scatter(x=x, y=y, name='calibration data', mode = 'markers', marker_color = px.colors.qualitative.Plotly[1], marker_symbol = 'circle-open', marker_line_width= 3))

# Create the trendline equation string
trendline_equation = f"T = {LR['slope']:.0f}\u00b1{LR['std_slope']:.0f} \u22c5 W + {LR['intercept']:.0f}\u00b1{LR['std_intercept']:.0f}<br>R<sup>2</sup> = {LR['r2']:.3f}"

# Add the trendline equation annotation
fig.add_annotation(
    go.layout.Annotation(
        x=0.95,
        xref = 'paper',
        yref='paper',
        y=0.1,
        text=trendline_equation,
        showarrow=False,
        font=dict(size=28),))
fig.update_traces(marker_size = 12)
fig.update_layout(xaxis_title = '<b>Power</b>, W',
                  yaxis_title = '<b>Temperature</b>, <sup>o</sup>C',
                  yaxis_title_font_color = px.colors.qualitative.Plotly[1],
                  yaxis_color = px.colors.qualitative.Plotly[1],
                  yaxis_side = 'right',
                  font_size = 22,
                  legend_x = 0.02,
                  legend_xanchor = 'left',
                  legend_y = 1,
                  yaxis_nticks = 5,
                  xaxis_nticks = 5,
                  yaxis_range = [18, 155],
                  xaxis_range = [-0.007, 0.615],
                  margin_l = 5,
                  height = 450,
                  width = 561)
fig.show()

In [ ]:
## Should be half of the Elsevier double column width at 300 dpi, which is 2244 / 2 = 1122
fig.write_image(f"../Graphs/Fig 2b. Pyrometer.png", format = 'png', scale=2)

### Fig 3. Protocol

total flow - 100 ml per min<br>
vessel conc - 100 ppm<br>
estimated dt = 100 ms<br>
sensor chamber volume = approx 20 ml<br>
tubing MFC - sensor chamber = up to 10 ml

In [ ]:
def bruteforce(data, slope, slope_std, intercept, intercept_std, repetitions: int = 1000):
    slopes = np.random.normal(slope, slope_std, repetitions)
    intercepts = np.random.normal(intercept, intercept_std, repetitions)
    boundaries = []
    for x in data:
        square = []
        for s in slopes:
            for i in intercepts:
                square.append(s*x + i)
        boundaries.append(np.quantile(a=square, q = [0.025, 0.975]).tolist())
    return boundaries

In [ ]:
df1 = load_data(gas).iloc[402:804, :]
power = df1.V.values * df1.I.values
tempCI = bruteforce(power, LR['slope'], LR['std_slope'], LR['intercept'], LR['std_intercept'], 500)
#tempCI

In [ ]:
np.append(np.array(tempCI)[:,0],np.array(tempCI)[:,1][::-1]).min()

In [ ]:
# figure 3.1 Single pulse that should match the dimensions of the pyrometer figure and should contain confidence interval
gas = 'NO2'
df1 = load_data(gas).iloc[402:804, :]
index = datetime_index_to_seconds(df1.index)
power = df1.V.values * df1.I.values
temperature = LR['slope'] * df1.V.values * df1.I.values + LR['intercept']
#temp_CI = bruteforce(power, LR['slope'], LR['std_slope'], LR['intercept'], LR['std_intercept'], 1000)

temp_df = pd.DataFrame(LR).iloc[:,1:3].set_index(df.index.values)
temp_df = temp_df.loc[temp_df.index < power.max()]
temp_index = np.append(index, index[::-1])
temp_CI = np.append(np.array(tempCI)[:,0],np.array(tempCI)[:,1][::-1])

fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(go.Scatter(x = index, y = df1.V, name = 'Voltage', line_color = px.colors.qualitative.Plotly[0]), secondary_y=True)
fig.add_trace(go.Scatter(x= index, y = temperature, name = 'Temp', line_color = px.colors.qualitative.Plotly[1]), secondary_y = False)
fig.add_trace(go.Scatter(x = temp_index, y = temp_CI, name = '95% CI', fill = 'toself', 
                         fillcolor=color_with_opacity(px.colors.qualitative.Plotly[1], 0.3),
                         line_color= 'rgba(0,0,0,0)'), secondary_y = False)
tolerances = print_tolerances(load_data(gas)).split('<br>')
fig.add_annotation(showarrow=False, xref='paper', x=0.2, y=35, text= tolerances[0], font_size = 28)
fig.add_annotation(showarrow=False, xref='paper', x=0.2, y=25, text= tolerances[1], font_size = 28)

fig.update_layout(yaxis_range = [11, 120.25],
                  yaxis_tickmode = 'array',
                  yaxis_tickvals = [30,50,70,90,110],
                  yaxis_title = '<b>Temperature</b>, <sup>o</sup>C',
                  yaxis_color = px.colors.qualitative.Plotly[1],
                  yaxis2_color = px.colors.qualitative.Plotly[0],
                  yaxis2_range = [0,27],
                  yaxis2_nticks = 7,
                  xaxis_tickmode= 'array',
                  xaxis_tickvals = [0, 10, 20, 30, 40, 50, 60],
                  xaxis_title= '<b>time</b>, s',
                  yaxis2_title= '<b>Voltage</b>, V',
                  xaxis_mirror = True,
                  legend_x = 0.03,
                  legend_xanchor = 'left',
                  legend_y = 1,
                  font_size = 22,
                  width = 561,
                  height= 450)

fig.show()

In [ ]:
# sizing: half of the Elsevier's full size 2244 / 2
fig.write_image(f"../Graphs/Fig 2c.tnr. One cycle.png", format = 'png', scale=2)

In [ ]:
# figure 3.2
gas = 'NO2'
df = load_data(gas).iloc[:50000, :]
print_tolerances(df, False)
gas_ind, gas_data = extract_gas_protocol(df)
gas_ind = datetime_index_to_seconds(pd.to_datetime(gas_ind), True)
index = datetime_index_to_seconds(df.index, True)
fig = go.Figure()

fig.add_trace(go.Scatter(x = index, y = df.V, name = 'Voltage'))

# add cycles count
for i in range(0, len(gas_ind) -1, 2):
    x_center = (gas_ind[i] + gas_ind[i+1]) / 2
    cycles_text = '10' if gas_data[i]==0 else '30 heating cycles'
    fig.add_annotation(showarrow=False,
                       text = cycles_text,
                       x= x_center,
                       y= 26.8)

fig.update_layout(xaxis_range = [-0.4,6287/60],
                  yaxis_range = [0,28.5],
                  yaxis_nticks = 7,
                  yaxis_side = 'right',
                  xaxis_tickmode= 'array',
                  xaxis_tickvals = [0, 20, 40, 60, 80, 100],
                  xaxis_title= '<b>time</b>, min',
                  yaxis_title= '<b>Voltage</b>, V',
                  shapes = create_gas_rectangles(gas_ind, gas_data),
                  margin_l = 5,
                  width = 1122,
                  height= 350)

fig.show()

In [ ]:
# figsize = double Elsevier 2244 pixels
fig.write_image('../Graphs/Fig 2a tnr One of 10 measurement loops_min.png', format='png', scale=2)

In [ ]:
df

#### Setup tolerances

In [ ]:
df.flow_target_error.describe(percentiles = [.0015, .005, .025, .975, .995, .9985]).apply(lambda x: round(x, 2))

In [ ]:
px.scatter(df.flow_target_error.iloc[150000:200000]).show()
# here up to 2 sec switching delay is observed
# CI 99.7 is [-0.57, 0.44] and when MFC should be closed it is closed. So, given that used flows are 10, 15, 25 sccm, the mistake is within 5% of MFC flow for 99.7% cases
# CI 99 is [-0.26, 0.19], so mistake is within 2% in 99% cases
# CI 95 is [0, 0.04], so mistake is within 0.4% in 95% cases
# overall MFC performs according to specs (very accurate) but there is a small delay of MFC switching
# given the 2 sec delay of switching and approx volume of tubing between MFC and sensor chamber 
# (up to 10 ml of tubing + 20ml chember itself => ping between MFC switch and sensor resistance change should be around 20 sec, since total flow is 100 ml/min)
# Given the fact that temp cycle duration is about 50 sec, the first cycle should always be discarded from analytical calculations
# But for recognition tasks it might be useful to include, since real data is noisy and in real world scenarios data cleaning might be inapplicable

In [ ]:
df.flow_carrier_error.describe(percentiles = [.0015, .005, .025, .975, .995, .9985]).apply(lambda x: round(x, 2))

In [ ]:
px.scatter(df.flow_carrier_error.iloc[150000:200000]).show()
# carrier MFC is even more accurate

### Fig 4. Raw data
Supplementary

In [ ]:
gas = 'H2S'
df = load_data(gas)
gas_ind, gas_data = extract_gas_protocol(df)
gas_ind = datetime_index_to_seconds(pd.to_datetime(gas_ind), True)
index = datetime_index_to_seconds(df.index, True)
fig = go.Figure()

fig.add_trace(go.Scatter(x = index, y = df.I, name = 'Current', hoverinfo='skip'))
fig.add_annotation(text= f'<b>{target_gas_dict[gas]}</b>', showarrow=False, xref='paper', xanchor='right', yref='paper', x=-0.01, y=1, font_size= 30)

fig.update_layout(shapes = create_gas_rectangles(gas_ind, gas_data, colorscale_dict[gas]),
                  yaxis_title= '<b>Current</b>, A',
                  margin_l= 50,
                  xaxis_title= '<b>Time</b>, min',
                  yaxis_range= [0.0005, 0.014],
                  xaxis_range= [0, 999],
                  width = 1122,
                  height= 400)

fig.show()

In [ ]:
fig.write_image(f'../Graphs/Fig. 4. Raw data {gas}.png', scale=2, format= 'png')

In [ ]:
fig = make_subplots(rows=3, shared_xaxes=True, vertical_spacing=0)

for row_n, gas, colorscale in zip([1,2,3],['NO2', 'H2S', 'Acet'],['Sunset', 'Plasma_r', 'Viridis_r']):
    df=load_data(gas)
    gas_ind, gas_data = extract_gas_protocol(df)
    gas_ind = datetime_index_to_seconds(pd.to_datetime(gas_ind), True)
    #if gas=='H2S':
    #    gas_ind, gas_data = gas_ind[1:], gas_data[1:]  # necessary if the start of H2S file is cut
    index = datetime_index_to_seconds(df.index, True)
    fig.add_trace(go.Scatter(x = index, y = df.I, name = target_gas_dict[gas], hoverinfo='skip'), row=row_n, col=1)
    for rect in create_gas_rectangles(gas_ind, gas_data, colorscale):
        fig.add_shape(rect, row=row_n, col=1)
    #fig.add_annotation(text= f'<b>{target_gas_dict[gas]}</b>', showarrow=False, xref='paper', xanchor='right', yref='paper', x=-0.03, y=1, font_size= 30)
fig.update_xaxes(mirror=True, row=1, col=1)
fig.update_xaxes(showticklabels = False, showline = False, ticklen = 0, row=1, col=1)
fig.update_xaxes(showticklabels = False, showline = False, ticklen = 0, row=2, col=1)
fig.update_layout(width = 1200,
                  height= 900,
                  #shapes=create_gas_rectangles(gas_ind, gas_data, colorscale_dict[gas]),
                  xaxis_range= [0,960],
                  xaxis3_mirror=False,
                  yaxis_range= [0.0005, 0.019],
                  yaxis2_range= [0.0005, 0.018],
                  yaxis3_range= [0.0005, 0.014],
                  xaxis3_title= '<b>Time</b>, min', 
                  yaxis2_title= '<b>Current</b>, A',
                  legend_x=1, legend_y = 1,
                  margin = dict(t=1, l=100, b=50))
    
fig.show()

In [ ]:
fig.write_image(f'../Graphs/Fig. 4. Raw data all gases.png', scale=2, format= 'png')

In [ ]:
fig = make_subplots(rows=2, cols=3, specs=[[{"colspan":3}, None, None],[{},{},{}]],
                    vertical_spacing=0.12, horizontal_spacing=0.03,
                    subplot_titles=('<b>a)                                                                                                                    NO<sub>2</sub></b> temperature envelopes', 
                                    '<b>b)                                     NO<sub>2</sub></b>', 
                                    '<b>c)                                     H<sub>2</sub>S</b>', 
                                    '<b>d)                                   Acetone</b>'))

gas = 'NO2'
df = load_data(gas)
gas_ind, gas_data = extract_gas_protocol(df)
gas_ind = datetime_index_to_seconds(pd.to_datetime(gas_ind), True)
index = datetime_index_to_seconds(df.index, True)

fig.add_trace(go.Scatter(x = index, y = df.I*1000, name = 'NO<sub>2</sub>', hoverinfo='skip', showlegend=False, line_color = px.colors.qualitative.Plotly[0]))
for rect in create_gas_rectangles(gas_ind, gas_data, colorscale_dict[gas], yref='y', y0=3.5, y1=18.9):
        fig.add_shape(rect, row=1, col=1)
for t in range(120, 30, -10):
    fig.add_trace(envelope(df, Temp= t, calib_slice_shift=15, is_mA = True))
fig.update_yaxes(range=[3.5, 18.9], title_text= '<b>Current</b>, mA', row=1, col=1)
#fig.update_xaxes(title_text = '<b>Time</b>, min')
fig.layout.annotations[0].update(font_size=30, x=0, xanchor='left')

# plotting the bottom row
for col_n, gas, colorscale, trace_color, xrange, yrange in zip([1,2,3],['NO2', 'H2S', 'Acet'],['Sunset', 'Plasma_r', 'Viridis_r'],px.colors.qualitative.Plotly[:3],[320, 220, 0],[18.17, 15.75, 13]):
    df=load_data(gas)
    gas_ind, gas_data = extract_gas_protocol(df)
    gas_ind = datetime_index_to_seconds(pd.to_datetime(gas_ind), True)
    #if gas=='H2S':
    #    gas_ind, gas_data = gas_ind[1:], gas_data[1:]  # necessary if the start of H2S file is cut
    index = datetime_index_to_seconds(df.index, True)
    fig.add_trace(go.Scatter(x = index, y = df.I*1000, name = target_gas_dict[gas], hoverinfo='skip', showlegend=False, line_color=trace_color), row=2, col=col_n)
    for rect in create_gas_rectangles(gas_ind, gas_data, colorscale, yref='y', y0=yrange, y1=yrange+0.6):
        fig.add_shape(rect, row=2, col=col_n)
    fig.update_xaxes(title_text= '<b>Time</b>, min',range=[xrange, xrange+350], row=2, col=col_n)
    fig.update_yaxes(tickangle = 0, range= [yrange, yrange+0.6], row=2, col=col_n)
    #fig.add_annotation(text= f'<b>{target_gas_dict[gas]}</b>', showarrow=False, xref='paper', xanchor='right', yref='paper', x=-0.03, y=1, font_size= 30)
fig.update_yaxes(title_text= '<b>Current</b>, mA', row=2, col=1)
fig.layout.annotations[1].update(font_size=30, xanchor='left', x=0, yanchor='top', y=0.5)  # updating the font_size of subplots
fig.layout.annotations[2].update(font_size=30, xanchor='left', x=0.345, yanchor='top', y=0.5)  # updating the font_size of subplots
fig.layout.annotations[3].update(font_size=30, xanchor='left', x=0.6875, yanchor='top', y=0.5)  # updating the font_size of subplots


#fig.update_xaxes(showticklabels = False, showline = False, ticklen = 0, row=2, col=1)
fig.update_layout(width = 2244,
                  height= 900,
                  legend_x=1, legend_y = 1,
                  margin = dict(t=50, l=12, r=12, b=12))
    
fig.show()

In [ ]:
fig.write_image(f'../Graphs/Fig. 4. woxaxtitle.png', scale=1, format= 'png')

### Fig 5. Raw data. envelopes

1. 3 subplots - rows

2. Gas depiction - either secondary y for every subplot, or create_rectangles

3. Envelope for each gas in the high plasma color

4. Multiple envelopes for NO2, multiple envelopes for H2S

In [ ]:
gas = 'H2S'
df = load_data(gas)
gas_ind, gas_data = extract_gas_protocol(df)
gas_ind = datetime_index_to_seconds(pd.to_datetime(gas_ind), True)
index = datetime_index_to_seconds(df.index, True)
fig = go.Figure()

fig.add_trace(go.Scatter(x = index, y = df.I, name = 'Current', hoverinfo='skip'))

'''for t in range(50, 120, 10):
    fig.add_trace(envelope(df, Temp= t))'''
fig.add_trace(envelope(df, V=25, calib_slice_shift=390))
fig.add_annotation(text= f'<b>{target_gas_dict[gas]}</b>', showarrow=False, xref='paper', xanchor='left', yref='paper', x=-0.122, y=1, font_size= 30)

fig.update_layout(shapes = create_gas_rectangles(gas_ind, gas_data, 'Sunset'),
                  yaxis_title= '<b>Current</b>, A',
                  margin_l= 140,
                  xaxis_title= '<b>Time</b>, min',
                  yaxis_range= [0.014, 0.0172],  # [0.0131, 0.01336],  #NO2 [0.0166, 0.0189], # Acet [0.002, 0.014],  # H2S [0.014, 0.0172],
                  #xaxis_range= [35, 350],
                  legend_y=1,
                  width = 1200,
                  height= 400)

fig.show()

In [ ]:
fig.write_image(f'../Graphs/Fig. 5.3 tnr Envelope {gas}.png', scale=2, format= 'png')

In [ ]:
gas = 'NO2'
df = load_data(gas)
gas_ind, gas_data = extract_gas_protocol(df)
gas_ind = datetime_index_to_seconds(pd.to_datetime(gas_ind), True)
index = datetime_index_to_seconds(df.index, True)
fig = go.Figure()

fig.add_trace(go.Scatter(x = index, y = df.I, name = 'Current', hoverinfo='skip'))

for t in range(40, 130, 20):
    fig.add_trace(envelope(df, Temp= t, calib_slice_shift=150))
fig.add_annotation(text= f'<b>{target_gas_dict[gas]}</b>', showarrow=False, xref='paper', xanchor='left', yref='paper', x=-0.12, y=1, font_size= 30)

fig.update_layout(shapes = create_gas_rectangles(gas_ind, gas_data, colorscale_dict[gas]),
                  yaxis_title= '<b>Current</b>, A',
                  margin_l= 140,
                  xaxis_title= '<b>Time</b>, min',
                  yaxis_range= [0.0025, 0.014], # NO2 [0.0035, 0.0189],  # Acet[0.002, 0.014],# [0.004, 0.0175], H2S
                  #xaxis_range= [0, 1100],
                  legend_y=1,
                  legend_xanchor='left',
                  width = 1200,
                  height= 400)

fig.show()

In [ ]:
fig.write_image(f'../Graphs/Fig. 5.2 tnr Envelopes {gas}.png', scale=2, format= 'png')

In [ ]:
gas= 'Acet'
fig = go.Figure()
fig.add_trace(go.Scatter(x= [1,2,2,3,3,4], y = [1,1,2.5,2.5,4,4], name= "conductance", mode='lines', showlegend=True))
fig.update_layout(shapes=create_gas_rectangles([1,2,2,3,3,4],[0,0,500,500,1000,1000], 'haline_r'),
                  yaxis_title= 'Analytical signal',
                  xaxis_title= 'Time',
                  title= 'Ideal sensor',)
fig.show()

### Fig 6. Raw data. Overlapping pulses
gas - air. Mean and 95%CI area plots

In [ ]:
gas = ['NO2', 'H2S', 'Acet'][0]
df = dedrift_n_cut(load_data(gas), 'multienv', 350, gas)
df.loc[:,gas] = df.loc[:,gas].apply(lambda x: round(x,0))
gas

In [ ]:
# H2S-specific tweaks
df = pd.concat([df.iloc[105:120,:], df], ignore_index=True)
df

In [ ]:
last_n_cycles = 8
z_score = 1.96
cheeky_ = subtract_previous_row(choose_last_n_cycles(df, last_n_cycles).iloc[:,:405]).iloc[last_n_cycles*2:,:]  # H2S tweak
mean1 = cheeky_.groupby(gas).mean(numeric_only = True).iloc[:,:402].T.drop(columns=0)
std1 = cheeky_.groupby(gas).std(numeric_only = True).iloc[:,:402].T.drop(columns=0)
x = mean1.index.values
colors = dict(zip(mean1.columns, px.colors.sample_colorscale('Sunset_r', minmax_scale(mean1.columns.values.tolist() + [0])[:-1])))
fig = go.Figure()

for col in mean1.columns:
    fig.add_trace(go.Scatter(x=x, y = mean1[col].values, name = f'{col:.0f} ppm', line_color = colors[col], legendgroup = col))
    fig.add_trace(trace_with_CI(index=x, trace=mean1[col].values, std=std1[col].values, z_score = z_score, fillcolor=colors[col], legendgroup = col))

fig.update_yaxes(showticklabels = False, showline = False, ticklen = 0)
fig.update_xaxes(mirror = False)
fig.update_layout(title = f'Thermal cycles: <br>difference between {target_gas_dict[gas]} cycle and the corresponding air cycle 30 cycles ago. <br>\
{calculate_p_value(z_score):.0f}% CI of {last_n_cycles} last cycles over each measurement protocol loop are plotted',
                  title_font_size = 25,
                  font_size = 25,
                  margin_t = 80,
                  xaxis_title = '# of the datapoint withing the cycle',
                  yaxis_title = "current, a.u.",  # <br>(dedrifted and normalized)
                  xaxis_range = [-2,407],
                  width = 1122)

fig.show()

In [ ]:
fig.write_image(f"../Graphs/Suppl/Fig 3. ({gas}_last{last_n_cycles}cycles_fancy_{calculate_p_value(z_score):.0f}CI).png", width = 1500, height = 500, scale = 2, format = 'png')

In [ ]:
last_n_cycles = 8
z_score = 1.96
df_ = choose_last_n_cycles(df, last_n_cycles)
mean = df_.groupby(gas).mean(numeric_only = True).iloc[:,:402].T.apply(lambda x: x- x[0], raw = True,axis = 1).drop(columns=0)
std_ = df_.groupby(gas).std(numeric_only = True).iloc[:,:402].T.drop(columns=0)
x = mean.index.values

fig = go.Figure()
colors = dict(zip(mean1.columns, px.colors.sample_colorscale('Sunset_r', minmax_scale(mean.columns.values.tolist() + [0])[:-1])))

for col in mean.columns:
    fig.add_trace(go.Scatter(x=x, y = mean[col].values, name = f'{col:.0f} ppm', line_color = colors[col], legendgroup = col))
    fig.add_trace(trace_with_CI(x, mean[col].values, std_[col].values, z_score=z_score, fillcolor=colors[col], legendgroup = col))

fig.update_yaxes(showticklabels = False, showline = False, ticklen = 0)
fig.update_xaxes(mirror = False)
fig.update_layout(title = f'Thermal cycles: difference between {target_gas_dict[gas]} cycles and the averaged air cycle <br>\
{calculate_p_value(z_score):.0f}% CI of {last_n_cycles} last cycles over each measurement protocol loop are plotted',
                  xaxis_title = '# of the datapoint withing the cycle',
                  yaxis_title = "current, a.u.",  #<br>(dedrifted and normalized)
                  xaxis_range = [-2,407],
                  title_font_size = 25,
                  font_size=25,
                  width = 1122)
fig.show()

In [ ]:
fig.write_image(f"../Graphs/Fig 6.2 ({last_n_cycles}_{calculate_p_value(z_score):.0f}CI).png", width = 1500, height = 500, scale = 2, format = 'png')

### Fig 7. Sensor signal

In [ ]:
def sensor_signal(I_air: float, I_gas: float) -> float:
    '''Calculates sensor signal'''
    ratio = I_air / I_gas if I_air > I_gas else I_gas / I_air
    return (ratio - 1)*100

In [ ]:
def temp_envelope(df: pd.DataFrame, Temp: float) -> pd.Series:
    '''Returns a pd Series with I data corresponding to the set `temp`
df should have I, V columns and timestamp index'''
    data = df.copy(deep=True)
    data = data.assign(temp = data.V * data.I * 202.7 + 28)  # T(W) calibration equation
    I_values_inv_T = []
    for cycle_n in range((data.shape[0] + 2) //402):  # because the last 2 datapoints are missing for NO2 data
        cycle = data.iloc[cycle_n*402: cycle_n*402 + 201,:]  # because one cycle containts 402 points - choose one half-cycle
        idx = (cycle.temp - Temp).abs().argsort()[:1].values[0]  # get the integer index of the cell with the temperature value closest to the Temp
        I_values_inv_T.append([cycle.index[idx], cycle.I.iloc[idx]])
    I_values_inv_T = np.array(I_values_inv_T)
    return pd.Series(data= I_values_inv_T[:,1], index= I_values_inv_T[:,0])

In [ ]:
def voltage_envelope(df: pd.DataFrame, datapoint: int = None, Voltage: float = None) -> pd.Series:
    '''Docstring'''
    if datapoint != None:
        target_slice = slice(datapoint, df.shape[0], 402)
    if Voltage:
        V_idx = (df.V.iloc[:201] - Voltage).abs().argsort()[:1].values[0]
        target_slice = slice(V_idx, df.shape[0], 402)
    return df.V.iloc[target_slice]   

In [ ]:
def SS_by_envelope(envelope: np.ndarray) -> np.ndarray:
    '''Docstring 10-30-10-30-10-30, 0-10-0-15-0-25'''
    if type(envelope) == pd.Series: envelope = envelope.values
    SS10, SS15, SS25 = [], [], []
    for i in range(10):  # number of measurement cycles
        if (gas=='Acet' and i==9): continue
        envelope_ = envelope[120*i: 120*(i + 1)]
        SS10.append(sensor_signal(envelope_[9], envelope_[39]))
        SS15.append(sensor_signal(envelope_[49], envelope_[79]))
        SS25.append(sensor_signal(envelope_[89], envelope_[119]))
    return np.array([SS10, SS15, SS25]).T

In [ ]:
SS_by_envelope(temp_envelope(load_data('NO2'), 201)).mean(axis=0)

In [ ]:
datapoint = 201
Temp = 120

SS_dict = {}
#colors = [px.colors.sequential.Sunset[-5], px.colors.sequential.Plasma[5], px.colors.sequential.haline[5]]
Temps = [45, 75, 105]
colors = {}
for gas, colorscale in zip(['NO2', 'H2S', 'Acet'], ['Sunset_r','Plasma', 'haline']):
    colors[gas] = dict(zip(Temps, px.colors.sample_colorscale(colorscale, minmax_scale(Temps + [0,125]))[:-2]))
fig = go.Figure()
for Temp in Temps:
    for gas, color, line_dash, marker_symbol in zip(['NO2', 'H2S', 'Acet'], colors, ['solid','longdash', 'dot'], ['square', 'triangle-up', 'circle']):
        SS_dict[gas] = SS_by_envelope(temp_envelope(load_data(gas), Temp=Temp))
        fig.add_trace(go.Scatter(x = [10,15,25], y= SS_dict[gas].mean(axis=0), error_y_array=SS_dict[gas].std(axis=0), 
                                 name= f'{Temp} <sup>o</sup>C', 
                                 marker_size = 15, 
                                 mode='lines+markers', 
                                 marker_color = colors[gas][Temp], 
                                 marker_symbol = marker_symbol,
                                 line_dash = line_dash,
                                 #showlegend=True if Temp == Temps[0] else False,
                                 legendgroup = gas, 
                                 legendgrouptitle_text = f'<b>{target_gas_dict[gas]}</b>'))
fig.update_layout(legend_y = 1,
                  legend_x = 0.025,
                  legend_xanchor='left', 
                  legend_orientation = 'h',
                  legend_title_font_size = 22,
                  legend_bgcolor = 'rgba(0,0,0,0)',
                  legend_itemwidth = 50,
                  yaxis_zeroline= True,
                  xaxis_title = '<b>Analyte concentration</b>, ppm',
                  yaxis_title= '<b>Sensor signal</b>, %',
                  yaxis_range= [-0.5, 3.4],
                  width = 561, 
                  height = 500,
                  )

fig.show()

In [ ]:
fig.write_image(f"../Graphs/Set 3/Fig 3. SS at {str(Temps)[1:-1]}C.png", format='png', scale=2)

Calculate sensor signal at different temperatures

Function for choosing appropriate envelopes

Add some averaging to reduce data noise? - probably no



### Fig 8. Response recovery time

In [ ]:
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import warnings
from scipy.optimize import OptimizeWarning
warnings.simplefilter(action='ignore', category=OptimizeWarning)

def exponential_func(t, Rmax, tau):
    return Rmax * (1 - np.exp(-t / tau))
def exponential_func_dec(t, Rmax, tau):
    return Rmax * np.exp(-t / tau)

def norm_data(data: np.ndarray) -> np.ndarray:
    data_ = data - data[0]
    return np.abs(data_)

def estimate_t90(data_: pd.Series) -> [float, float, float]:
    time = datetime_index_to_seconds(data_.index)
    data = norm_data(data_.values)
    t90_func = lambda tau: tau*np.log(10)
    #fitting data
    R_max_guess = data.max()
    tau_guess = R_max_guess * (time[4] - time[0]) / (data[4] - data[0])
    popt, pcov = curve_fit(exponential_func, time, data, [R_max_guess, tau_guess])
    Rmax_fit, tau_fit = popt
    #estimating uncertainty
    Rmax_uncertainty, tau_uncertainty = np.sqrt(np.diag(pcov))
    t90 = t90_func(tau_fit)
    partial_deriv_tau = np.log(10)
    t90_uncertainty = np.abs(partial_deriv_tau * tau_uncertainty)  #/t90
    #calculating r2
    predicted_values = exponential_func(time, Rmax_fit, tau_fit)
    r_squared = r2_score(data, predicted_values)

    return [t90, t90_uncertainty, r_squared]

In [ ]:
def env_into_pieces(data: pd.Series) -> (np.ndarray, np.ndarray):
    '''Takes the envelope in pd.Series form and cuts it into array of series of individual segments'''
    response, recovery = [], []
    for i in range(10):  # number of measurement cycles
        if (gas=='Acet' and i==9): continue
        env = data.iloc[i*120:(i+1)*120]
        # recovery response 10 15 25 
        response.append([env.iloc[10:40], env.iloc[50:80], env.iloc[90:120]])
        recovery.append([env.iloc[40:50], env.iloc[80:90], env.iloc[:10]])
    return response, recovery

In [ ]:
gas = 'H2S'
Temp = 120
resp, rec = env_into_pieces(temp_envelope(load_data(gas), Temp))
data = rec[5][2]  # third meas_cycle and 25 ppm
time = datetime_index_to_seconds(data.index)
data = norm_data(data.values)
t90_func = lambda tau: np.abs(tau*np.log(10))
#fitting data
R_max_guess = data.max()
tau_guess = R_max_guess * (time[4] - time[0]) / (data[4] - data[0])  
popt, pcov = curve_fit(exponential_func, time, data, [R_max_guess, tau_guess])
Rmax_fit, tau_fit = popt
#estimating uncertainty
Rmax_uncertainty, tau_uncertainty = np.sqrt(np.diag(pcov))
t90 = t90_func(tau_fit)
t90_uncertainty = t90_func(tau_uncertainty)  #/t90
#calculating r2
predicted_values = exponential_func(time, Rmax_fit, tau_fit)
R2 = r2_score(data, predicted_values)

from IPython.display import display, HTML
from plotly.offline import init_notebook_mode
## Tomas Mazak's workaround
init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))
##
fig = go.Figure()
fig.add_trace(go.Scatter(x = time, y = data, name= f'{target_gas_dict[gas]}<br>raw data', mode='markers'))
fig.add_trace(go.Scatter(x = time, y = predicted_values, name= 'fit', mode='lines'))
fig.add_annotation(xref='paper', yref='paper', x=0.05, y=0.95, showarrow=False, 
                   text=r'$I(t) = {I}_{max} \cdot (1 - 10^{- \frac{t}{{\tau}_{\tiny 90}}})$')
fig.add_annotation(xref='paper', yref='paper', x=0.05, y=0.85, showarrow=False, 
                   text=f'I<sub>max</sub> = {Rmax_fit:.2e} \u00b1 {Rmax_uncertainty / Rmax_fit * 100:.0f}% [A]')
fig.add_annotation(xref='paper', yref='paper', x=0.05, y=0.75, showarrow=False, 
                   text=f'\u03c4<sub>90</sub> = {t90_func(tau_fit):.0f} \u00b1 {tau_uncertainty / tau_fit * 100:.0f}% [s]')
fig.add_annotation(xref='paper', yref='paper', x=0.05, y=0.60, showarrow=False, 
                   text=f'R<sup>2</sup> = {R2:.3f}')
fig.update_layout(legend_y = 0.1,
                  xaxis_title = 'time, sec',
                  yaxis_title= 'Current, A',
                  height = 350, width = 1000,
                  )
fig.show()

In [ ]:
fig.write_image(f'../Graphs/Fig 9.1 {gas} at {Temp} C.png', format='png', scale=2)

In [ ]:
def pieces_into_traces(datapieces: list, gas: str, is_resp: bool) -> go.Scatter:
    '''takes the list of lists of pd.Series as datapieces, fits every segment of data, returns the go.Scatter trace of response_time vs. concentration'''
    concentrations = [10,15,25]
    text = 'response' if is_resp else 'recovery'
    line_dash = 'solid' if is_resp else 'dash'
    marker_pattern_shape = None if is_resp else "x"
    color_dict = dict(zip('NO2 H2S Acet'.split(), [px.colors.sequential.Sunset_r[4], px.colors.sequential.Plasma[5], px.colors.sequential.haline[5]]))
    color_error_dict = dict(zip('NO2 H2S Acet'.split(), [px.colors.sequential.Sunset_r[6], px.colors.sequential.Plasma[7], px.colors.sequential.haline[7]]))
    gas_data = np.zeros(shape=(len(datapieces), len(datapieces[0]), 3))  # 3 on the end because estimate_t90 has 3 outputs - t90, t90_std and R2
    for meas_cycle in range(len(datapieces)):
        for conc in range(len(concentrations)):
            t90, t90_std, R2 = estimate_t90(datapieces[meas_cycle][conc])
            if t90_std / t90 > 0.2 or R2 < 0.9:
                t90, t90_std, R2 = np.NaN, np.NaN, np.NaN  # if the t90_std is too large (above 20% of t90) or R2 is below 0.9 - discard the result
            gas_data[meas_cycle][conc] = [t90, t90_std, R2]
    return go.Bar(x= concentrations, 
                      y= np.nanmean(gas_data[:,:,0], axis=0), 
                      error_y_array= np.nanmean(gas_data[:,:,1], axis=0),
                      error_y_color = color_error_dict[gas],
                      error_y_thickness = 3,
                      #mode='lines',
                      name=f'{text}',
                      legendgroup= gas,
                      legendgrouptitle_text= target_gas_dict[gas],
                      marker_color = color_dict[gas],
                      marker_pattern_shape = marker_pattern_shape,
                      #color = color_dict[gas], 
                      #line_dash = line_dash, 
                      legendwidth = 80)

def pieces_into_multitraces(datapieses_resp: list, datapieces_rec: list, gas: str) -> list:
    '''takes resp and rec returns the list of go.Scatter'''
    resp = pieces_into_traces(datapieses_resp, gas, True)
    rec = pieces_into_traces(datapieces_rec, gas, False)
    return [resp, rec]

In [ ]:
Temp = 110
traces = []
for gas in 'NO2 H2S Acet'.split():
    traces_ = pieces_into_multitraces(*env_into_pieces(temp_envelope(load_data(gas), Temp)), gas)
    traces.extend(traces_)
fig = go.Figure(data=traces)
fig.add_annotation(xref='paper', yref='paper', x=0.02, y=0.9875, showarrow=False, 
                   text=f'<b>{Temp:.0f} <sup>o</sup>C</b>', font_size = 25)
fig.update_layout(yaxis_title = '<b>Response time</b>, sec',
                  xaxis_title = '<b>Analyte concentration</b>, ppm',
                  legend_orientation = 'h',
                  legend_font_size = 15,
                  legend_xanchor = 'right',
                  legend_x = 1,
                  legend_y = 1,
                  legend_bgcolor = 'rgba(255,255,255,0.05)',
                  yaxis_range= [200, 2500],
                  #yaxis_type = 'log',
                  width = 561,
                  height = 500,
                  )
fig.show()

In [ ]:
fig.write_image(f'../Graphs/Set 3/Fig 3a Response_recovery at the {Temp:.0f}.png', scale=2, format= 'png')

# Raman

In [ ]:
raman_raw_files = [r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Raman 2023\CNT_KZ_laser532_0.5mW_exposure6Hz_slit50cp_1.1.txt",
                   r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Raman 2023\CNT_KZ_laser532_0.5mW_exposure6Hz_slit50cp_1.2.txt",
                   r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Raman 2023\CNT_KZ_laser532_0.5mW_exposure6Hz_slit50cp_1.3.txt",
                   r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Raman 2023\CNT_KZ_laser532_0.5mW_exposure6Hz_slit50cp_1.4.txt",
                   r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Raman 2023\CNT_KZ_laser532_0.5mW_exposure6Hz_slit50cp_1.5.txt"]
df = []
for i, file in enumerate(raman_raw_files):
    data = pd.read_csv(file, sep='\t', header=None, names= ['Index', 'Intensity'])
    if not i:
        df.append(data.Index.values.tolist())
    df.append(data.Intensity.values.tolist())
df = pd.DataFrame(data=np.array(df).T, columns= 'Raman_shift 1 2 3 4 5'.split())
df.set_index('Raman_shift', inplace=True)
df = df.apply(lambda x: x / max(x), raw= True)
df.mean(axis=1)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df.index, y = df.mean(axis=1), name='free-standing SWCNT', showlegend=True, line_width = 3))


# Add annotations for D-mode, G-mode, and RBM peaks with maximum values
for peak_position, peak_name, ax, ay in [(1346.2, 'D mode', -90, -20), (1596.9, 'G mode', -100, 30), (2684.5, '2D mode', -80, -10)]:
    
    fig.add_annotation(
        go.layout.Annotation(
            x=peak_position,
            y=df.mean(axis=1).loc[peak_position-0.5: peak_position + 0.5].values[0],
            xref="x",
            yref="y",
            text=f"<b>{peak_name}</b> <br>{peak_position:.1f} cm<sup>-1</sup>",
            showarrow=True,
            arrowhead=2,
            ax=ax,
            ay=ay
        )
    )
fig.update_layout(
    width = 800,
    height = 400,
    yaxis_tickmode='array',
    yaxis_tickvals=[],
    yaxis_title = '<b>Intensity</b>, a.u.',
    xaxis_title = '<b>Raman shift</b>, cm<sup>-1</sup>',
    legend_y = 1,
    legend_title = '',
    margin_t = 0)

In [ ]:
fig.write_image(f'../Graphs/Set 3/Fig 2 Raman_mean_.svg', format= 'svg')

## examples from other notebooks

#### Sensor signal calculation
example from plasma.humidity report

In [ ]:
def calc_SS(series, start, minutes_plasma: float, ptype_response: bool, plm: int = 10, purge_duration_min: int = 5, return_full_info: bool =False, correct_baseline: bool = True):
    '''plm = plus minus x seconds when searching for 
       start = seconds before protocol start
       ptype_response = True if resistance in NO2 drops. If None then True if series.name > 9'''
    if ptype_response == None:
        ptype_response = True if minutes_plasma > 9 else False
    R_air, R_gas = [], []
    SS, bc_SS = [], []
    p = purge_duration_min*60
    for i in range(21):
        R_air_index = slice(start+p*(2*i+1)-plm,start+p*(2*i+1)+plm)
        R_gas_index = slice(start+2*p*(i+1)-plm,start+2*p*(i+1)+plm)
        if ptype_response:
            R_air.append(series.loc[R_air_index].max())
            R_gas.append(series.loc[R_gas_index].min())
            SS.append((R_air[i]-R_gas[i])/R_gas[i]*100)
        else:
            R_air.append(series.loc[R_air_index].min())
            R_gas.append(series.loc[R_gas_index].max())
            SS.append((R_gas[i]-R_air[i])/R_air[i]*100)
            
    for i in range(20):
        if ptype_response:
            R_a = (R_air[i+1] + R_air[i])/2
            bc_SS.append((R_a-R_gas[i])/R_gas[i]*100)
        else:
            R_a = (R_air[i+1] + R_air[i])/2
            bc_SS.append((R_gas[i]-R_a)/R_a*100)
    bc_SS.append(np.NaN)
    #LOD ss|50ppm = 3*std
    #print(f"LOD = {stds[series.name]}")
    if return_full_info:
        return pd.DataFrame(data = zip(R_air, R_gas, SS, bc_SS), index = np.arange(start+p*2, start+p*2*22, p*2), columns = "R_air R_gas SS bc_SS".split())
    else:
        if correct_baseline:
            return bc_SS
        else: 
            return SS
def smooth_df(df, **savgol_params):
    index = df.index.values
    new_df, mape_, std_ = [], [], []
    for i in range(4):
        smooth = savgol_filter(df.iloc[:, i], **savgol_params)
        new_df.append(smooth)
        std = [(x-y)**2 for x,y in zip(df.iloc[:, i],smooth)]
        std_.append(np.sqrt(np.sum(std)/len(std)))
        mape = np.nanmean([np.abs(true - sm)/sm*100 for true, sm in zip(df.iloc[:, i],smooth)])
        mape_.append(mape)
        print(f"Mape for {df.columns[i]} = {mape:.4f} %")
    return pd.DataFrame(zip(*new_df), index = index, columns = df.columns), dict(zip(df.columns[:4],mape_)), dict(zip(df.columns[:4],std_))
def plot_RR(df, column_n: int = 3, start_s: int = 50, calc_SS_raw: bool = True, MovAver: bool = False, **savgol_params):
    #initial
    RH = int(df.columns.name.split()[1])
    sample_names = [0,7.5,12,17]
    if start_s == None:
        shift_dict = {'RH 0': 47, 'RH 30': 49, 'RH 50': 54}
        start_s = shift_dict[df.columns.name]
    
    # smoothing
    smooth_ = df.iloc[:, column_n].rolling(window = savgol_params['window_length'], center=True).mean() if MovAver else savgol_filter(df.iloc[:, column_n], **savgol_params)
    smooth = pd.DataFrame(data = zip(smooth_, df.iloc[:,column_n].values.tolist()), index = df.index, columns = ["smooth", "raw"])
    RMSE = mean_squared_error(smooth.dropna().smooth.values.tolist(), smooth.dropna().raw.values.tolist(), squared = False)
    
    # Sensor signal
    SS = calc_SS(df.iloc[:,column_n] if calc_SS_raw else smooth.smooth, 
                 start = start_s, 
                 minutes_plasma = sample_names[column_n],
                 ptype_response = None, 
                 plm = 10, purge_duration_min = 5, return_full_info = True)
    mean_stable_SS = SS.iloc[5:,3].mean()
    LOD_p = RMSE/SS.iloc[10:,0].mean()*300
    LOD_ppm = LOD_p/mean_stable_SS*50
    SS.index.name = str(f"{RH} {sample_names[column_n]} ") + str(int(round(LOD_ppm*1000)))
    
    print(f"LOD = {LOD_ppm:.3f} ppm ({LOD_p:.3f}%), bcSS = {SS.iloc[:,3].mean():.3f} \u00b1 {SS.iloc[:,3].std():.3f}")
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = df.index.values, y = df.iloc[:,column_n], name = f'сырые данные. <br>Образец <b>{df.columns[column_n]}</b>', line_color='blue'))
    fig.add_trace(go.Scatter(x = df.index.values, y = smooth.smooth, name = f'сглаженные данные. <br>{savgol_params}', line_color='orangered'))
    fig.add_trace(go.Scatter(x = [i - 300 for i in SS.index.values], y = SS.iloc[:,0], name = f'<b>R<sub>air</sub></b>: воздух, влажность {RH}%', 
                             mode = 'markers', marker_color = 'lime', marker_size=15, marker_symbol='diamond'))
    fig.add_trace(go.Scatter(x = SS.index.values, y = SS.iloc[:,1], name = f'<b>R<sub>gas</sub></b>: NO<sub>2</sub> 50 млн<sup>-1</sup> + воздух, влажность {RH}%', 
                             mode = 'markers', marker_color = 'orange', marker_size=15, marker_symbol='diamond'))
    fig.update_layout(template = 'simple_white',
                      font_size = 22,
                      legend_font_size = 15,
                      xaxis_title = '<b>Время</b>, сек',
                      yaxis_title = '<b>Сопротивление</b>, Ом',
                      yaxis_nticks = 4)
    fig.show()
    return SS.iloc[:,2], SS.iloc[:,3], LOD_ppm

#### Pulses grouping example

In [ ]:
# Clever legend grouping
# this code allows to plot all the lines from big preprocessed dataframe 
# using the same color for conc and hiding unnecessary legends
def plot_I_pulses_grouped(DF, gas: str = 'NO2', save: bool = False):
    fig = go.Figure()
    
    labels = DF.iloc[-5,:].unique()
    conc_to_color_dict = {labels[i]: range(len(labels))[i] for i in range(len(labels))}
    show_in_legend_dict = {labels[i]: True for i in range(len(labels))}
    
    for row in range(len(DF.columns)):
        conc = DF.iloc[-5,col]
        color = px.colors.qualitative.Plotly[conc_to_color_dict[conc]]
        name = str(f"{round(conc,0)} ppm")
        fig.add_trace(go.Scattergl(x= np.arange(DF.shape[0] - 1),  # DF.index[:-1],
                                 y= DF.iloc[:-1,col],
                                 name = name,
                                 mode = 'lines',
                                 legendgroup = name,
                                 showlegend = show_in_legend_dict[conc],
                                 line=dict(width=1,color=color)))
        show_in_legend_dict[conc] = False
    fig.update_layout(legend_traceorder='grouped',
                      title=dict(text=str(f"I(t) curves, {target_gas_dict[target_gas]} measurements"), 
                                 font=dict(size=30),
                                 xref="paper",
                                 xanchor="auto",
                                 x=0.85),
                      xaxis=dict(visible=True),
                      yaxis=dict(title=dict(text='<b>I</b>, A')),
                      plot_bgcolor = 'white')
    if save:
        date = dt.today().strftime("_%d-%m_%H-%M")
        fig.write_html(str(f"C:/Users/Konstantin_Z/OneDrive/SK/zzz____LNM/_Experiments/_New SWCNT measurements/{gas}{date}.html"))
    fig.show()
    return None

In [ ]:
SS = pd.read_csv('Sensor signal_top_last_cycle_acetcut.txt', sep = '\t', decimal = ',')

In [ ]:
SSd = pd.read_csv('Sensor signal_top_last_cycle_acetcut-drift.txt', sep = '\t', decimal = ',')

In [ ]:
fig = go.Figure(layout = dict(template = 'simple_white',
                              title_text = 'Sensor signal',  #<br>drift compensated with lowess (frac = 0.3)<br>exact formula: new_i = data_i - lowess_i + lowess_mean',
                              xaxis_nticks = 5,
                              xaxis_title_text = '<b>Concentration</b>, ppm',
                              yaxis_nticks = 5,
                              yaxis_title_text = '<b>Sensor signal</b>, %',
                              width = 700,
                              height = 700))
fig.add_trace(go.Scatter(x = SS.index, y = SS.H2S, error_y_array = SS.H2S_err, name = 'H<sub>2</sub>S', line_color = px.colors.qualitative.Dark2[0]))
fig.add_trace(go.Scatter(x = SSd.index, y = SSd.H2S, error_y_array = SSd.H2S_err, name = 'H<sub>2</sub>S - drift', line_color = px.colors.qualitative.Set2[0]))

fig.add_trace(go.Scatter(x = SS.index, y = SS.Acet, error_y_array = SS.Acet_err, name = 'Acetone', line_color = px.colors.qualitative.Dark2[1]))
fig.add_trace(go.Scatter(x = SSd.index, y = SSd.Acet, error_y_array = SSd.Acet_err, name = 'Acetone - drift', line_color = px.colors.qualitative.Set2[1]))

fig.add_trace(go.Scatter(x = SS.index, y = SS.NO2, error_y_array = SS.NO2_err, name = 'NO<sub>2</sub>', line_color = px.colors.qualitative.Dark2[2]))
fig.add_trace(go.Scatter(x = SSd.index, y = SSd.NO2, error_y_array = SSd.NO2_err, name = 'NO<sub>2</sub> - drift', line_color = px.colors.qualitative.Set2[2]))
fig.show()

### Raman

In [ ]:
import os
from pathlib import Path
#data_storage_path = 'C:/Users/Konstantin_Z/OneDrive/SK/zzz____LNM/_Experiments/_New SWCNT measurements/Response time meas/'
data_storage_path = 'C:/Users/Bahrs/YandexDisk/SK_LNM/zzz____LNM/_Experiments/Raman/Raman data/'
#data_storage_path = 'C:/Users/zamba_000/OneDrive/SK/zzz____LNM/_Experiments/Raman/Raman data/'
#data_storage_path = 'D:/Константин/OneDrive/SK/zzz____LNM/_Experiments/Raman/Raman data/'
data_list = [str(x)[len(data_storage_path):] for x in sorted(Path(data_storage_path).iterdir(), key=os.path.getmtime, reverse=True)]
data_files_list = [data_storage_path + x for x in data_list]
data_list_df = pd.DataFrame(data=data_list, columns = ['file names'], index = np.arange(len(data_list)))
data_list_df

In [ ]:
df = pd.read_csv(data_files_list[0], 
                 sep = ';',
                 decimal = ',',
                 )
df

In [ ]:
fig = px.line(df, x = df.columns[0], y = df.columns[[1,2,3,4,5,6,7,8,9]],
              color_discrete_sequence= px.colors.sequential.Plasma,
              title = "Original Raman spectroscopy data <br>traces in legend are subscripted according to the voltage applied to the sample during Raman measurements"
       )
fig.show()

In [ ]:
fig.write_html("D:/Константин/OneDrive/SK/zzz____LNM/_Experiments/Raman/Interactive graphs/Raman-original.html")

#### Data processing

##### fitting with 30 point cubic Savitzky-Golay filter (nearest)

In [ ]:
dfsg = pd.DataFrame()
dfsg["Raman shift"] = df["Raman shift"]
for i in range(1, len(df.columns)):
    dfsg[str(df.columns[i])] = savgol_filter(x = df[df.columns[i]].to_list(), window_length = 25, polyorder = 3, mode='nearest')
dfsg

In [ ]:
fig = px.line(dfsg, x = dfsg.columns[0], y = dfsg.columns[[1,2,3,4,5,6,7,8,9]],
          color_discrete_sequence= px.colors.sequential.Plasma,
          title = "Raman data, filtered with SavGol filter (window_length=25, polyorder=3, mode=nearest) <br>\
          traces in legend are subscripted according to the voltage applied to the sample during Raman measurements",
          )
fig.show()

In [ ]:
fig.write_html("D:/Константин/OneDrive/SK/zzz____LNM/_Experiments/Raman/Interactive graphs/Raman-SavGol-25-3.html")

### response recovery

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess
def smooth(data, smoothing_factor: float = 0.1):
    if type(data) == list or type(data) == np.ndarray:    
        x = np.arange(0,len(data))*5
        lowess_data = lowess(data, x, frac=smoothing_factor)[:,1]
        return lowess_data
    elif type(data) == pd.Series:
        lowess_data = lowess(data.values, data.index, frac=smoothing_factor)[:,1]
        return pd.Series(data=lowess_data, index = data.index)
    else: 
        print(f"Wrong data type {type(data)}")

In [ ]:
sf = 0.2
sample = 8
samples = dict(enumerate(DF.columns.tolist()))
y = DF.iloc[:,sample].values
smoothed = smooth(y, sf)
fig = go.Figure()
fig.add_trace(go.Scatter(x = DF.index, y = y, name = 'raw'))
fig.add_trace(go.Scatter(x = DF.index, y = smoothed, name = f'lowess {sf:.1f}'))
fig.show()

y -= smoothed
x = DF.index.values[:240]
cut_data_response, cut_data_recovery = [], []
fig = go.Figure()
color = sample_colorscale('plasma', minmax_scale(np.arange(20)))
for gas_purge in range(20):
    response_slice = slice(int(gas_purge*480+240),int((gas_purge+1)*480))
    recovery_slice = slice(int((gas_purge+1)*480),int((gas_purge+1)*480+240))
    cut_data_response.append(normalize_cut_data(y[response_slice]))
    cut_data_recovery.append(normalize_cut_data(y[recovery_slice]))
    fig.add_trace(go.Scatter(x = x, y = cut_data_recovery[-1], line_color = color[gas_purge], name = str(gas_purge)))

fig.update_layout(template = 'simple_white',
                  height = 800,
                  title = f"{samples[sample]:.1f} min sample recovery NO<sub>2</sub> 50 ppm. ",  # Drift corr: lowess {sf:.1f}
                  yaxis_title = 'Normalized resistance R-R<sub>0</sub>',
                  xaxis_title = 'Time, s',
                  font_size = 30,
                  legend_font_size = 20)
fig.show()

### Response recovery times

In [ ]:
# DF = NO2, DF1 = NH3
cut_data_response, cut_data_recovery = [], []
for gas_purge in range(20):
    response_slice = slice(int(gas_purge*480+240),int((gas_purge+1)*480))
    recovery_slice = slice(int((gas_purge+1)*480),int((gas_purge+1)*480+240))
    cut_data_response.append(DF.iloc[response_slice,:].values.T.tolist())
    cut_data_recovery.append(DF.iloc[recovery_slice,:].values.T.tolist())
cut_data_response = np.transpose(np.array(cut_data_response), (1,2,0))
cut_data_recovery = np.transpose(np.array(cut_data_recovery), (1,2,0))
print(cut_data_recovery.shape)
#print(cut_data_recovery[0][0,:])


In [ ]:
def normalize_cut_data(data: list):
    return [x-data[0] for x in data]
def normalize_columns(data: np.ndarray):
    first_values = data[0,:]
    return data - first_values
def custom_minmax(data):
    max, min = data.max(), data.min()
    minmax = lambda x: (x - min)/(max - min)
    return np.vectorize(minmax)(data)

#### Overlapping transients graphs

In [ ]:
fig = go.Figure()
samples = dict(enumerate(DF.columns.tolist()))
sample = 5
cycles = 20
color = sample_colorscale('plasma', minmax_scale(np.arange(cycles)))
for cycle in range(cycles):
    y = smooth(normalize_cut_data(cut_data_recovery[sample,:,cycle]), 0.1)  
    #y = [np.log(i) for i in normalize_cut_data(cut_data_response[cycle][sample])]
    x = np.arange(0,len(y))*5
    fig.add_trace(go.Scatter(x=x, y=y, mode = "lines", line_color=color[cycle], name = cycle))
fig.update_layout(template = 'simple_white',
                    height = 800,
                  title = f"{samples[sample]:.1f} min sample recovery NO<sub>2</sub> 50 ppm",
                  yaxis_title = 'Normalized resistance R-R<sub>0</sub>',
                  xaxis_title = 'Time, s',
                  font_size = 30,
                  legend_font_size = 20)
fig.show()

In [ ]:
fig = go.Figure()
samples = dict(enumerate(DF.columns.tolist()))
samples_ = [3,4,5,6,7,8]
#sample = 8
cycles = 20
color = sample_colorscale('haline', minmax_scale(np.arange(cycles)))
line_dash = dict(zip(samples_, 'solid longdash longdashdot dash dashdot dot'.split()[:len(samples_)]))
samples_title = ''
for sample in samples_:
    # minmax the stuff
    data = custom_minmax(np.abs(normalize_columns(cut_data_recovery[sample])))
    samples_title += f"{samples[sample]:.1f}, "
    for cycle in range(cycles):
        y = smooth(data[:,cycle], 0.1)  # data[:,cycle]  # 
        x = np.arange(0,len(y))*5
        showtrace_in_legend = True if cycle == 0 else False
        fig.add_trace(go.Scatter(x=x, y=y, mode = "lines", line_color=color[cycle], name = '', line_dash = line_dash[sample], 
                                 legendgroup=str(sample), legendgrouptitle_text=str(samples[sample]), showlegend = showtrace_in_legend))
samples_title = samples_title[:-2]
colorbar_ticks = [0,5,10,15,19]
fig.add_trace(
    go.Scatter(
        x=[None], y=[None], mode='markers', showlegend = False,
        marker=dict(colorbar=dict(title="gas purge cycle #",
                                  titleside="top",
                                  tickmode="array",
                                  tickvals=minmax_scale(colorbar_ticks, feature_range=(1,7)),  # (2,7)
                                  ticktext=[str(a) for a in colorbar_ticks],
                                  tickfont_size=30,
                                  ticks="outside",
                                  orientation='h',
                                  yanchor="top",
                                  len = 0.975,
                                  y=0.95,
                                  x=0.01,
                                  xanchor="left",
                                  ),
                    colorscale=[[a/19,b] for a, b in zip(colorbar_ticks,[color[i] for i in colorbar_ticks])]),))
fig.update_layout(height = 800,
                  width = 1400,
                  yaxis_range = [0,1.4],
                  template = 'simple_white',
                  title = f"Samples recovery NO<sub>2</sub> 50 ppm",
                  title_x = 0.52,
                  yaxis_title = '<b>Normalized rescaled resistance<b>',   # r'$\huge{\text{Normalized resistance}\frac{R - R_0}{R_{max} - R_{min}}}$',
                  yaxis_nticks = 5,
                  xaxis_title = '<b>Time,</b> s',
                  font_size = 30,
                  xaxis_mirror=True,
                  yaxis_mirror=True,
                  xaxis_linewidth=2,
                  yaxis_linewidth=2,
                  legend_font_size = 20,
                  legend_orientation = 'h',
                  legend_y = 0.68, legend_x = 0.01, legend_itemwidth = 80)
fig.show()

In [ ]:
fig_save(fig, 'Response time NO2 trends rescale norm', height = 800, width = 1400, scale = 3)

In [ ]:
#fig.write_image(file = f'sample {samples[sample]:.1f} min responses to NO2.svg', format ='svg')
graph_savepath = "/".join(r'C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Plasma treatment\Article graphs\Response times'.split("\\"))
print(graph_savepath)
fig.write_html(f"{graph_savepath}/{samples_title} NO2 50 ppm.html")

#### Fitting cut data

##### Debugging

In [ ]:
from scipy.optimize import curve_fit

# Define the exponential function
def exponential_func_inc(t, Rmax, tau):
    return Rmax * (1 - np.exp(-t / tau))
def exponential_func_dec(t, Rmax, tau):
    return Rmax * np.exp(-t / tau)

# Generate example raw data (replace with your actual data)
time = np.arange(240) * 5
raw_data = pd.Series(normalize_cut_data(cut_data_recovery[5][:,10]), index=time)

# Define the function to calculate the derived parameter
def t90(tau):
    return tau * np.log(10)


# Perform the fit
popt, pcov = curve_fit(exponential_func, time, raw_data)

# Extract the fitted parameters and their uncertainties
Rmax_fit, tau_fit = popt
Rmax_uncertainty, tau_uncertainty = np.sqrt(np.diag(pcov))

# Calculate the derived parameter
derived_param = t90(tau_fit)

# Calculate the uncertainty in the derived parameter using error propagation formula sqrt((d_param/d_param1*sigma_param1)**2 + ...)
partial_deriv_tau = np.log(10)

# Compute the uncertainty using error propagation formula
derived_param_uncertainty = np.abs(partial_deriv_tau * tau_uncertainty)

# Print the results
print(f"Rmax = {Rmax_fit} \u00b1 {Rmax_uncertainty}, tau = {tau_fit} \u00b1 {tau_uncertainty}")
print("Derived Parameter:", derived_param)
print("Uncertainty in Derived Parameter:", derived_param_uncertainty/derived_param*100, "%")

fig = go.Figure()


##### Actual calculation

In [ ]:
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

def exponential_func(t, Rmax, tau):
    return Rmax * (1 - np.exp(-t / tau))

def estimate_t90(data):
    time = np.arange(len(data)) * 5  # sampling interval 5s
    data = np.abs(data) 
    t90_func = lambda tau: tau*np.log(10)
    #fitting data
    popt, pcov = curve_fit(exponential_func, time, data)
    Rmax_fit, tau_fit = popt
    #estimating uncertainty
    Rmax_uncertainty, tau_uncertainty = np.sqrt(np.diag(pcov))
    t90 = t90_func(tau_fit)
    partial_deriv_tau = np.log(10)
    t90_uncertainty = np.abs(partial_deriv_tau * tau_uncertainty)  #/t90
    #calculating r2
    predicted_values = exponential_func(time, Rmax_fit, tau_fit)
    r_squared = r2_score(data, predicted_values)

    return [t90, t90_uncertainty, r_squared]

def load_rrt_data(is_NO2: bool, is_resp: bool):
    cut_data_response, cut_data_recovery = [], []
    df = DF if is_NO2 else DF1
    for gas_purge in range(20):
        response_slice = slice(int(gas_purge*480+240),int((gas_purge+1)*480))
        recovery_slice = slice(int((gas_purge+1)*480),int((gas_purge+1)*480+240))
        cut_data_response.append(df.iloc[response_slice,:].values.T.tolist())
        cut_data_recovery.append(df.iloc[recovery_slice,:].values.T.tolist())
    cut_data_response = np.transpose(np.array(cut_data_response), (1,2,0))
    cut_data_recovery = np.transpose(np.array(cut_data_recovery), (1,2,0))
    if is_resp:
        return cut_data_response
    else: 
        return cut_data_recovery
    
def calc_t90_df(is_NO2: bool, is_resp: bool):
    t90_ = np.zeros((cut_data_recovery.shape[0], cut_data_recovery.shape[2], 3))
    for sample in range(len(DF.columns)):
        for cycle in range(cut_data_response.shape[2]):
            data = normalize_cut_data(load_rrt_data(is_NO2, is_resp)[sample][:,cycle])
            t90_[sample, cycle] = estimate_t90(data)
    t90_clean = np.zeros((cut_data_recovery.shape[0], cut_data_recovery.shape[2], 3))
    t90_df = np.zeros((9,2))
    for sample in range(len(DF.columns)):
        for cycle in range(20):
            if t90_[sample, cycle, 2] > 0.95 and t90_[sample, cycle, 1]/t90_[sample,cycle,0] < 0.3:  # and t90_[sample, cycle, 1]/t90_[sample,cycle,0] < 1
                t90_clean[sample,cycle] = t90_[sample,cycle]
            else: t90_clean[sample,cycle] = [np.NaN, np.NaN, np.NaN]
        t90_df[sample] = [np.nanmean(t90_clean[sample,:,0]), np.nanstd(t90_clean[sample,:,0])]
    return t90_df

    

In [ ]:
gas = 'NO2'
response = False
t90_ = np.zeros((cut_data_recovery.shape[0], cut_data_recovery.shape[2], 3))
for sample in range(len(DF.columns)):
    for cycle in range(cut_data_response.shape[2]):
        data = normalize_cut_data(load_rrt_data((gas == 'NO2'), is_resp=response)[sample][:,cycle])
        t90_[sample, cycle] = estimate_t90(data)

In [ ]:
t90_clean = np.zeros((cut_data_recovery.shape[0], cut_data_recovery.shape[2], 3))
t90_df = np.zeros((9,2))
for sample in range(len(DF.columns)):
    for cycle in range(20):
        if t90_[sample, cycle, 2] > 0.95 and t90_[sample, cycle, 1]/t90_[sample,cycle,0] < 0.05:  # and t90_[sample, cycle, 1]/t90_[sample,cycle,0] < 1
            t90_clean[sample,cycle] = t90_[sample,cycle]
        else: t90_clean[sample,cycle] = [np.NaN, np.NaN, np.NaN]
    t90_df[sample] = [np.nanmean(t90_clean[sample,:,0]), np.nanstd(t90_clean[sample,:,0])]
t90_df

In [ ]:
fig = go.Figure()
title_resp = 'Response' if response else 'Recovery'
yaxis_resp = 'response' if response else 'recovery'
gas_ = 'NO<sub>2</sub>' if gas == 'NO2' else 'NH<sub>3</sub>'
color = sample_colorscale('plasma', minmax_scale(DF.columns.tolist() + [20]))
for sample in range(len(DF.columns)):
    fig.add_trace(go.Scatter(x = np.arange(20)+1, y = t90_clean[sample,:,0], 
                             error_y_array = t90_clean[sample,:,1], error_y_type='data', error_y_thickness = 3,
                             line_color = color[sample], line_width = 4,
                             name = f"{DF.columns[sample]:.1f}"))  #  R<sup>2</sup>={np.nanmean(t90_clean[sample,:,2]):.2f}
fig.add_annotation(xref='paper',
                    yref='paper',
                    xanchor='right',
                    yanchor='bottom',
                    x=-0.055,
                    y=1.065,
                    text=f" <b>b</b> ",
                    font_size=60,
                    showarrow=False,
                    borderwidth=1,
                    bordercolor='rgba(0,0,0,0)')

fig.update_layout(template = 'simple_white', 
                  margin = {'t':110, 'l':150, 'r':10, 'b':0},
                  title = f"{title_resp} time vs gas purge cycle number. {gas_} 50 ppm",
                  title_x = 0.1,
                  xaxis_title = '<b>gas purge number</b>, #', 
                  yaxis_title = f'<b>{yaxis_resp} time</b>, s',
                  font_size = 30, legend_font_size = 30, legend_orientation = 'h', legend_y = 1, legend_x = 0.05,
                  #yaxis_range = [0, 5300],
                  xaxis_range = [0, 20.5], 
                  yaxis_nticks = 5, 
                  xaxis_nticks = 5,
                  height = 800, 
                  width = 1400,
                  xaxis_linewidth = 2,
                  yaxis_linewidth = 2,
                  xaxis_mirror = True,
                  yaxis_mirror = True,)
annotation_text = str(f"Average {yaxis_resp} time is {round(np.nanmean(t90_df[:,0]/60), 0):.0f} \u00b1 {round(np.nanmean(t90_df[:,1]/60), 0):.0f} minutes")
fig.add_annotation(text = annotation_text, x = 0.05, y = 0.9, xref = 'paper', yref = 'paper', showarrow=False)
fig.show()

In [ ]:
# NO2 best resp rec
print(f"Average {gas} {yaxis_resp} time for 18 min sample is {round(np.nanmean(t90_clean[7, :, 0])/60, 0):.0f} \u00b1 {round(np.nanstd(t90_clean[7, :, 0])/60, 0):.0f} minutes")
print(f"Average {gas} {yaxis_resp} time for 15 min sample is {round(np.nanmean(t90_clean[4, :, 0])/60, 0):.0f} \u00b1 {round(np.nanstd(t90_clean[4, :, 0])/60, 0):.0f} minutes")

#### Continue the data_fitting

In [ ]:
from lmfit import Model
def sigmoid(x, L, k, x0, y0):
  sigmoid = L/(1+np.exp(-k*(x-x0))) + y0
  return sigmoid
def inverse_sigmoid(y, L, k, x0, y0):
  return 1/k*np.log((y-y0)/(L-y+y0)) + x0

def fit_sigmoid(raw: list, dt: float = 5, plot: bool = False):
  time_sec = np.arange(0,len(raw))*dt
  model = Model(sigmoid)
  model.set_param_hint('L', value = max(raw[0], raw[-1]), min=0, max = raw[-1]*2)
  model.set_param_hint('k', value = (raw[20]-raw[0])/5/21)
  model.set_param_hint('x0', value = 0)
  model.set_param_hint('y0', value = 0 if raw[0] < raw[-1] else raw[-1])
  params = model.make_params()
  result = model.fit(raw, params, x = time_sec)
  print(result.fit_report())
  best_params = result.best_values
  if plot:  
    dely = result.eval_uncertainty(x=time_sec)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = time_sec, y = raw, name='raw_data', mode = 'markers'))
    fig.add_trace(go.Scatter(x = time_sec, y = result.best_fit, name=f'fitted_data', mode = 'lines'))
    fig.add_trace(go.Scatter(x = time_sec, y = result.best_fit + dely, name='upper_bound', mode = 'lines', marker_color="#444", line_width = 0))  # showlegend = False,
    fig.add_trace(go.Scatter(x = time_sec, y = result.best_fit - dely, name='lower_bound', mode = 'lines', marker_color="#444", line_width = 0, fill = 'tonexty', fillcolor = 'rgba(100, 100, 100, 0.3)'))
    fig.update_layout(title = str(best_params))
    fig.show()
  target_value = 0.9*(best_params['L']-raw[0])+raw[0] if raw[0] < raw[-1] else raw[0] - 0.9*abs(best_params['L']-raw[0])
  print(best_params['L'], target_value)
  t90 = inverse_sigmoid(target_value, **best_params)
  print(t90)
  y_fit = sigmoid(time_sec, **best_params)
  return t90, result.rsquared, y_fit

In [ ]:
easy_function_list = [(func, func.__name__, inv_func) for func, inv_func in 
                      zip([exponential, power_law, Michaelis_Menten, Hill_eq], [inv_exponential, inv_power_law, inv_MM, inv_Hill_eq])]

In [ ]:
def easy_fit(y_data, func, inv_func, plot: bool):
    x_data = np.arange(0,len(y_data))*5  # assuming 5 sec between datapoints
    model = Model(func)
    model.set_param_hint('Rmax', value = max(y_data[0], y_data[-1]), min=0, max = y_data[-1]*3)
    params = model.make_params()
    result = model.fit(y_data, params, t = x_data)
    best_params = result.best_values
    increasing = bool(y_data[0] < y_data[-1])
    target_value = 0.9*(best_params['Rmax']-y_data[0])+y_data[0] if increasing else y_data[0] - 0.9*abs(best_params['Rmax']-y_data[0])
    t90 = inv_func(target_value, **best_params)
    print(result.fit_report())
    if plot:
        dely = result.eval_uncertainty(x=x_data)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x = x_data, y = y_data, name='raw_data', mode = 'markers'))
        fig.add_trace(go.Scatter(x = x_data, y = result.best_fit, name=f'fitted_data', mode = 'lines'))
        fig.add_trace(go.Scatter(x = x_data, y = result.best_fit + dely, name='upper_bound', mode = 'lines', marker_color="#444", line_width = 0))  # showlegend = False,
        fig.add_trace(go.Scatter(x = x_data, y = result.best_fit - dely, name='lower_bound', mode = 'lines', marker_color="#444", line_width = 0, fill = 'tonexty', fillcolor = 'rgba(100, 100, 100, 0.3)'))
        fig.update_layout(title = str(best_params))
        fig.show()
    R2 = result.rsquare
    return t90, R2, result.best_fit
    

In [ ]:
from scipy.optimize import curve_fit
def scipyfit(y_data, func, inv_func, plot: bool = False):
    time_sec = np.arange(0,len(y_data))*5
    #y0_guess = 0 if y_data[0] < y_data[-1] else y_data[-1]
    #p0 = [max(y_data[0], y_data[-1]), (y_data[20]-y_data[0])/1.6/21, 0, y0_guess]
    popt, pcov = curve_fit(func, time_sec, y_data, maxfev=1500)  ## bounds for L (supremum) p0 = p0, bounds=([0, -np.inf, -np.inf,-y_data[0]*3],[np.inf, np.inf, 0, y_data[-1]*3])
    y_fit = [func(x,*popt) for x in time_sec]
    if plot:  
        fig = go.Figure()
        fig.add_trace(go.Scatter(x = time_sec, y = y_data, name='y_data_data', mode = 'markers'))
        fig.add_trace(go.Scatter(x = time_sec, y = y_fit, name=f'fitted_data<br>{popt}', mode = 'lines'))
        fig.update_layout()
        fig.show()
    target_value = 0.9*(popt[0]-y_data[0])+y_data[0] if y_data[0] < y_data[-1] else y_data[0] - 0.9*abs(popt[0]-y_data[0])
    print(target_value)
    t90 = inv_func(target_value, *popt)
    return t90, y_fit

In [ ]:
data = normalize_cut_data(cut_data_recovery[1][5])
x_data = np.arange(0,len(data))*5
fig = go.Figure()
fig.add_trace(go.Scatter(x = x_data, y=data, name = 'raw', mode = 'markers'))
for func, func_name, inv_func in easy_function_list:
    t90, y_fit = scipyfit(data, func, inv_func, plot=False)
    fig.add_trace(go.Scatter(x = x_data, y = y_fit, mode = 'lines', name = func_name+str(f' {t90:.3f}')))
fig.show()

In [ ]:
data = normalize_cut_data(cut_data_recovery[2][7])
x_data = np.arange(0,len(data))*5
fig = go.Figure()
fig.add_trace(go.Scatter(x = x_data, y=data, name = 'raw', mode = 'markers'))
for func, func_name, inv_func in easy_function_list:
    t90, R2, y_fit = easy_fit(data, func, inv_func, plot=True)
    fig.add_trace(go.Scatter(x = x_data, y = y_fit, mode = 'lines', name = func_name+str(f' {t90:.3f} {R2:.3f}')))
fig.show()

In [ ]:
data = normalize_cut_data(cut_data_recovery[1][6])
x_data = np.arange(0,len(data))*5
fig = go.Figure()
fig.add_trace(go.Scatter(x = x_data, y=data, name = 'raw', mode = 'markers'))
t90, R2, y_fit = fit_data(data, dt=5, plot=True)
fig.add_trace(go.Scatter(x = x_data, y = y_fit, mode = 'lines', name = func_name+str(f' {t90:.3f} {R2:.3f}')))
fig.show()

In [ ]:
sample = 8
sample_dict = dict(zip(np.arange(9),DF.columns.tolist()))
data = normalize_cut_data(cut_data_recovery[2][sample][6:])
x_data = np.arange(0,len(data))*5
model = Model(s_exp)
initial_tau = data[-1]/((data[20]-data[0])/5/21)
model.set_param_hint('Rmax', value = max(data[0], data[-1]), min=-abs(data[-1]*3), max = abs(data[-1]*3))
model.set_param_hint('tau', value = initial_tau, min = initial_tau/5, max=initial_tau*5)
#model.set_param_hint('x0', value = 0)
#model.set_param_hint('y0', value = 0 if data[0] < data[-1] else data[-1])
params = model.make_params()
result = model.fit(data, params, t = x_data)
print(result.fit_report())
best_params = result.best_values 
target_value = 0.9*(best_params['Rmax']-data[0])+data[0] if data[0] < data[-1] else data[0] - 0.9*abs(best_params['Rmax']-data[0])
t90 = inv_s_exp(target_value, **best_params)
Rmax_perr= result.fit_report().split("[[")[3].split("\n")[1].split("%)")[0].split("(")[-1]
tau_perr = result.fit_report().split("[[")[3].split("\n")[2].split("%)")[0].split("(")[-1]
dely = result.eval_uncertainty(t=x_data)
fig = go.Figure()
fig.add_trace(go.Scatter(x = x_data, y = data, name='data_data', mode = 'markers'))
fig.add_trace(go.Scatter(x = x_data, y = result.best_fit, name=f'fitted_data', mode = 'lines'))
fig.add_trace(go.Scatter(x = x_data, y = result.best_fit + dely, name='upper_bound', mode = 'lines', marker_color="#444", line_width = 0))  # showlegend = False,
fig.add_trace(go.Scatter(x = x_data, y = result.best_fit - dely, name='lower_bound', mode = 'lines', marker_color="#444", line_width = 0, fill = 'tonexty', fillcolor = 'rgba(100, 100, 100, 0.3)'))
fig.update_layout(title = str(f"Sample {sample_dict[sample]} min <br>") + str(best_params) + str(f"\t Rmax {Rmax_perr}%, tau {tau_perr}%\t R<sup>2</sup> = {result.rsquared:.3f}<br>t<sub>90</sub> = {int(t90/60)} min {int(t90)%60} sec"))
fig.show()

#print(best_params['Rmax'], target_value)
print(result.fit_report().split("[[")[3].split("\n")[1].split("%)")[0].split("(")[-1], result.fit_report().split("[[")[3].split("\n")[2].split("%)")[0].split("(")[-1])
print(t90/60)

# Playground

In [ ]:
gas = 'H2S'  # NO2 H2S Acet
df = load_data(gas)

In [ ]:
quick_I_plot(df, df.columns[1]).show()

## Plan

1. Add cutting to pulses function
2. Compare last pulse from each group
3. Add a function to export and plot certain envelopes (already realized in Data Processing notebook)
4. Find the code for calculating t_resp t_recovery

1. Raw data

2. Raw data but similar to Ti article - algorithm for choosing the last pulses

2.1. algorithm for pulses normalization

2.2. calculate and plot mean single cycle resistace

2.3. calculate and plot standard deviation

3. sensor signal, sensor signal - drift

4. calculate the response recovery time with the code above. Maybe repeat for different transients

List of graphs for the article

1. Sensor setup - suppementary?

2. Pyrometer calibration / Raman

3. Protocol (voltage, temperature_pyrometer, temperature_modelling)

4. Raw data overview

5. Raw data - envelopes

6. Raw data - overlapping pulses

7. Sensor signal

8. Response / recovery time throughout the measurements (similar to plasma article)

9. Tables reporting ML stuff

# Pyrometer

In [ ]:
df = pd.read_csv(r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Pyrometer and Raman\Pyrometer.txt", sep='\t', decimal=',', index_col=0)

In [ ]:
fig = px.scatter(df, trendline = 'ols')
ols_results = px.get_trendline_results(fig)
slope = ols_results.iloc[0]["px_fit_results"].params[1]
intercept = ols_results.iloc[0]["px_fit_results"].params[0]
rsquared = ols_results.iloc[0]["px_fit_results"].rsquared

# Create the trendline equation string
trendline_equation = f'y = {slope:.1f}x + {intercept:.1f}<br>R<sup>2</sup> = {rsquared:.3f}'

# Add the trendline equation annotation
fig.add_annotation(
    go.layout.Annotation(
        x=0.95,
        xref = 'paper',
        yref='paper',
        y=0.1,
        text=trendline_equation,
        showarrow=False,
        font=dict(size=25),
    )
)
fig.update_traces(marker_size = 12)
fig.update_layout(template = 'simple_white',
                  xaxis_title = '<b>Power</b>, W',
                  yaxis_title = '<b>Temperature</b>, <sup>o</sup>C',
                  showlegend = False,
                  margin = dict(l=50, t=10, r=10, b=10),
                  font_size = 20,
                  yaxis_nticks = 5,
                  xaxis_nticks = 5,
                  height = 500,
                  width = 500)
fig.show()

In [ ]:
fig.write_image(r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Pyrometer and Raman\Pyrometer.png", format = 'png', scale=2)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_scatter_with_trendline_and_ci(x, y):
    # Create a scatter plot with trendline and confidence interval
    sns.regplot(x=x, y=y, ci=95, scatter_kws={'s': 50})
    
    # Customize the plot
    #sns.set(rc={'figure.figsize':(18,6)})
    plt.title('Scatter Plot with Trendline and 95% Confidence Interval')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    
    plt.show()
x = df.index.values
y = np.array(df.T.values.tolist()[0])
plot_scatter_with_trendline_and_ci(x, y)

# Raman

In [ ]:
df = pd.read_csv('C:/Users/Bahrs/YandexDisk/SK_LNM/zzz____LNM/_Experiments/Raman/Raman data/summary.csv', sep = ';', decimal = ',')
dfsg = pd.DataFrame()
dfsg["Raman shift"] = df["Raman shift"]
for i in range(1, len(df.columns)):
    dfsg[str(df.columns[i])] = savgol_filter(x = df[df.columns[i]].to_list(), window_length = 25, polyorder = 3, mode='nearest')

In [ ]:
max_Raman_shift = dfsg.iloc[dfsg.idxmax()[1:].values.tolist(),0].values.tolist()
max_intensity = dfsg.max()[1:].values.tolist()

In [ ]:
max_Raman_shift[0]

In [ ]:
fig = px.line(dfsg, x = dfsg.columns[0], y = dfsg.columns[[1,2,3,4,5,6,7,8,9]],
          color_discrete_sequence= px.colors.sequential.Plasma,
          )
fig.add_trace(go.Scatter(x=max_Raman_shift, y=max_intensity, mode='markers', 
                         marker_size = 10, marker_symbol='circle-dot', marker_color = 'lightskyblue', marker_line_color = 'midnightblue', marker_line_width = 2,
                         showlegend=False))
fig.add_vline(x = max_Raman_shift[0], line_color = 'lightskyblue', line_width =2)
fig.update_yaxes(showticklabels = False)
fig.update_layout(template = 'simple_white',
                  xaxis_title = '<b>Raman shift</b>, cm<sup>-1</sup>',
                  yaxis_title = '<b>Intensity</b>, a.u.',
                  legend_title_text = 'Voltage',
                  legend_bgcolor = 'rgba(256,256,256,0)',
                  legend_x = 1.0,
                  legend_orientation = 'v',
                  legend_xanchor = 'right',
                  legend_yanchor = 'top',
                  legend_y = 1,
                  xaxis_range=[1550, 1610],
                  yaxis_range=[850,15500],
                  margin = dict(l=50, t=0, r=0, b=10),
                  font_size = 20,
                  yaxis_nticks = 1,
                  xaxis_nticks = 5,
                  height = 500,
                  width = 500)
fig.show()

In [ ]:
fig.write_image(r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Pyrometer and Raman\Raman.png", format = 'png', scale=2)

In [ ]:
max_Raman_shift

In [ ]:
fig = px.scatter(x = max_Raman_shift, y=[int(x[:-1]) for x in dfsg.columns[1:]], trendline = 'ols')
ols_results = px.get_trendline_results(fig)
slope = ols_results.iloc[0]["px_fit_results"].params[1]
intercept = ols_results.iloc[0]["px_fit_results"].params[0]
rsquared = ols_results.iloc[0]["px_fit_results"].rsquared

# Create the trendline equation string
trendline_equation = f'y = {slope:.1f}x + {intercept:.1f}<br>R<sup>2</sup> = {rsquared:.3f}'

# Add the trendline equation annotation
fig.add_annotation(
    go.layout.Annotation(
        x=0.95,
        xref = 'paper',
        yref='paper',
        y=0.1,
        text=trendline_equation,
        showarrow=False,
        font=dict(size=25),
    )
)
fig.update_traces(marker_size = 12)
fig.update_layout(template = 'simple_white',
                  xaxis_title = '<b>Raman shift</b>, cm<sup>-1</sup>',
                  yaxis_title = '<b>Voltage</b>, V',
                  showlegend = False,
                  margin = dict(l=50, t=0, r=0, b=10),
                  font_size = 20,
                  yaxis_nticks = 5,
                  xaxis_nticks = 5,
                  height = 500,
                  width = 500)

In [ ]:
fig.write_image(r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Pyrometer and Raman\Raman_calib.png", format = 'png', scale=2)